In [1]:
import time
import pandas as pd
import logging
import numpy as np
import pandas as pd
from pyxlsb import open_workbook as open_xlsb
from os import listdir
from os.path import isfile, join
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 32)
import shutil
import os
import datetime
import sys
sys.path.append('../src/')
from final_script_risk_management import transform_entity_stat, transform_others, create_sum_by_sf, final_calculation, get_examples
import warnings
warnings.filterwarnings("ignore")
#logging.basicConfig(level=logging.DEBUG, filename="prospecting_tool.log", filemode='a', format='%(asctime)s - %(message)s')

In [2]:
#df_base= pd.read_excel(r"../input/Prospect Analytics business case Model.xlsx" , sheet_name = 'Template')
map_match= pd.read_excel('../input/mapping_of_matching_category.xlsx')
rep_assignment= pd.read_excel('../input/activity_status.xlsx', sheet_name='Rep Assignment')
entity_status= pd.read_excel('../input/entity_status.xlsx')
df_role= pd.read_excel(r"../input/UserswithRoles.xlsx" , sheet_name = 'Role_Mapping')
df_prospects= pd.read_excel('../input/Prospects.xlsx')
path_match=''
book= pd.read_excel('../input/columns_required.xlsx')
cols = list(book['columns'])

In [3]:
df= pd.read_excel(r"../input/Prospect Tracking 2019.xlsx" , sheet_name = 'Template for Sumit')

In [4]:
#set(list(df.columns))- set(list(df_base.columns))

In [5]:
# df = df.rename(columns= {'Public Record 2019 ': 'Public Record 2016', '% change 2019 vs. 2018': '% change 2017 vs. 2016', 
#                         'Public Record 2018': 'Public Rdcord 2018', 'Public Record 2019 ':'Public Record 2016'})

In [6]:
df_base= df

In [7]:
df.shape
print(len(df[df['Parent name (per 2018 PR)'].notnull()]))
len(df[df['Parent name (per 2019 PR)'].notnull()])

1547


1472

In [8]:
from datetime import date

Date = date.today()
Date = Date.strftime("%b-%d-%Y")
print(Date)

Jul-20-2020


In [9]:
df_base = df_base[df_base['Parent name (per 2018 PR)'].notnull()] 

df_base.rename(columns = {'Self Filing share %': 'SF share of public filing',
                             'Auto (new & used) loan %': 'Auto (New & used) loan %', 
                             'WKLS share %': 'WKLS share of public filing',
                             '# of filing zips / Name Variations': '# of filing offices (/ # of filing zips)',
                             '# of filing states w/ filing > 100': '# of jurisdictions w/ filing > 100',
                             '# of filing states': '# of filing jurisdictions'}, inplace = True)

df_base['Sales assignemnt'] = df_base['Sales assignemnt'].str.replace('?','Unknown')
duplicate = (df_base[df_base.duplicated()])
# print(duplicate.shape)
# print("There are  %s number of duplicate values are there in our dataset out of %s"  %(duplicate, len(df_base)))

In [10]:
df_base = df_base.drop_duplicates(keep='first', inplace=False)
df_base.shape

(1547, 36)

In [11]:
print(df_base['Index'].nunique())
print(df_base['Index'].isna().sum())
df_base['Index'].fillna('Index-unknown' , inplace = True)
print(df_base['Index'].isna().sum())

1422
125
0


In [12]:
df_base.isna().sum()

Index                                                  0
Client (PAG)                                        1547
Parent name (per 2018 PR)                              0
Parent name (per 2019 PR)                             75
Public Record 2019                                   125
Public Record 2018                                    18
% change 2019 vs. 2018                               169
SF share of public filing                            139
WKLS share of public filing                          139
% of filing using non WKLS vendors                   139
# of filing offices (/ # of filing zips)             125
UCC 3 % over total filing                            125
# of filing jurisdictions                             77
# of jurisdictions w/ filing > 100                    77
Top vol state                                        125
2nd vol state                                        142
3rd vol state                                        309
Top UCC vol State Srch rating  

In [13]:
print(df_base['Public Record 2019 '].isna().sum())
df_base['Public Record 2019 '] =  df_base['Public Record 2019 '].fillna(0).astype(int)
print(df_base['Public Record 2019 '].isna().sum())

print(df_base['Public Record 2018'].isna().sum())
df_base['Public Record 2018'] = df_base['Public Record 2018'].fillna(0).astype(int)
df_base['Public Record 2018'].isna().sum()

125
0
18


0

In [14]:
#df.info()
df_base['% change 2019 vs. 2018']=df_base['% change 2019 vs. 2018'].fillna(0)
df_base['% change 2019 vs. 2018'] = df_base['% change 2019 vs. 2018'].apply(lambda x: round(x,2)*100).astype(int)

In [15]:
#df.info()
df_base['SF share of public filing']=df_base['SF share of public filing'].fillna(0)
df_base['WKLS share of public filing']=df_base['WKLS share of public filing'].fillna(0)
df_base['% of filing using non WKLS vendors']=df_base['% of filing using non WKLS vendors'].fillna(0)

df_base['SF share of public filing'] = df_base['SF share of public filing'].apply(lambda x: round(x,2)*100).astype(int)
df_base['WKLS share of public filing'] = df_base['WKLS share of public filing'].apply(lambda x: round(x,2)*100).astype(int)
df_base['% of filing using non WKLS vendors'] = df_base['% of filing using non WKLS vendors'].apply(lambda x: round(x,2)*100).astype(int)

In [16]:
df_base['# of filing offices (/ # of filing zips)']=df_base['# of filing offices (/ # of filing zips)'].fillna(0)
df_base['UCC 3 % over total filing']=df_base['UCC 3 % over total filing'].fillna(0)
df_base['# of filing jurisdictions']=df_base['# of filing jurisdictions'].fillna(0)
df_base['# of jurisdictions w/ filing > 100']=df_base['# of jurisdictions w/ filing > 100'].fillna(0)

df_base['# of filing offices (/ # of filing zips)'] = df_base['# of filing offices (/ # of filing zips)'].apply(lambda x: round(x,2)).astype(int)
df_base['UCC 3 % over total filing'] = df_base['UCC 3 % over total filing'].apply(lambda x: round(x,2)*100).astype(int)
df_base['# of filing jurisdictions'] = df_base['# of filing jurisdictions'].apply(lambda x: round(x,2)).astype(int)
df_base['# of jurisdictions w/ filing > 100'] = df_base['# of jurisdictions w/ filing > 100'].apply(lambda x: round(x,2)).astype(int)

In [17]:
df_base['Top vol state'].unique()
df_base['2nd vol state'].unique()
df_base['3rd vol state'].unique()

array(['MA', 'CO', nan, 'WI', 'NY', 'IL', 'CA', 'KY', 'KS', 'NE', 'NV',
       'MT', 'FL', 'AK', 'AR', 'MO', 'IA', 'OK', 'DE', 'AL', 'OH', 'TX',
       'TN', 'DC', 'AZ', 'NC', 'NJ', 'MD', 'MS', 'ID', 'OR', 'GA', 'SC',
       'MI', 'WV', 'CT', 'MN', 'WY', 'ME', 'PA', 'UT', 'WA', 'SD', 'NM',
       'VT', 'IN', 'LA', 'RI', 'VA', 'ND', 'HI', 'NH'], dtype=object)

In [18]:
df_base['Top vol state'] = df_base['Top vol state'].fillna('XX')
df_base['2nd vol state'] = df_base['2nd vol state'].fillna('XX')
df_base['3rd vol state'] = df_base['3rd vol state'].fillna('XX')
df_base['Updated User type']=df_base['Updated User type'].fillna('')

In [19]:
df_base['Parent name (per 2019 PR)']=df_base['Parent name (per 2019 PR)'].fillna('')

In [20]:
df_base['Top UCC vol State Srch rating']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1542   NaN
1543   NaN
1544   NaN
1545   NaN
1546   NaN
Name: Top UCC vol State Srch rating, Length: 1547, dtype: float64

In [21]:
df_base['Top UCC vol State Srch rating'].unique()
df_base['2nd UCC vol State Srch rating'].unique()
df_base['3rd UCC vol State Srch rating'].unique()

array([nan])

In [22]:
#df_base.columns

In [23]:
df_base['3rd UCC vol State Srch rating'].fillna("XXX" , inplace = True)
df_base['Top UCC vol State Srch rating'].fillna("XXX" , inplace = True)
df_base['2nd UCC vol State Srch rating'].fillna("XXX" , inplace = True)

In [24]:
#% of filing vol in top 6 states (how disbursed?)
df_base['% of filing vol in top 6 states (how disbursed?)']=df_base['% of filing vol in top 6 states (how disbursed?)'].fillna(0)
df_base['% of filing vol in top 6 states (how disbursed?)'] = df_base['% of filing vol in top 6 states (how disbursed?)'].apply(lambda x: round(x,2)*100).astype(int)

In [25]:
df_base['Self Filer Segments'].unique()
df_base['Self Filer Segments'] = df_base['Self Filer Segments'].fillna("Unknown")
df_base['Self Filer Segments'].unique()
df_base['Self Filer Segments'].value_counts()

Bank                 776
Others               343
Credit Union         142
Unknown              131
FCS                   56
Equipment Finance     38
0                     21
Consumer Lender       20
Government            13
Solar                  3
MERS                   1
Deere                  1
Holding company        1
USDA/FSA/CCC           1
Name: Self Filer Segments, dtype: int64

In [26]:
df_base['Asset size (M)'].fillna(0 , inplace = True )
df_base['Commercial loan value (M)'].fillna(0 , inplace = True )
df_base['Commercial loans %'].fillna(0 , inplace = True )
df_base['Consumer loans %'].fillna(0 , inplace = True )
df_base['Real estate loan %'].fillna(0 , inplace = True )
df_base['Auto (New & used) loan %'].fillna(0 , inplace = True )

In [27]:
df_base['Commercial loans %'] = df_base['Commercial loans %'].apply(lambda x: round(x,2)*100).astype(int)
df_base['Consumer loans %'] = df_base['Consumer loans %'].apply(lambda x: round(x,2)*100).astype(int)
df_base['Real estate loan %'] = df_base['Real estate loan %'].apply(lambda x: round(x,2)*100).astype(int)
df_base['Auto (New & used) loan %'] = df_base['Auto (New & used) loan %'].apply(lambda x: round(x,2)*100).astype(int)

In [28]:
df_base['Personal Lien %'].fillna(0 , inplace = True )
df_base['Business Lien%'].fillna(0 , inplace = True )
df_base['% Filing Date Order > 1'].fillna(0 , inplace = True )
df_base['Personal Lien %'] = df_base['Personal Lien %'].apply(lambda x: round(x,2)*100).astype(int)
df_base['Business Lien%'] = df_base['Business Lien%'].apply(lambda x: round(x,2)*100).astype(int)
df_base['% Filing Date Order > 1'] = df_base['% Filing Date Order > 1'].apply(lambda x: round(x,2)*100).astype(int)

In [29]:
df_base['Delinquency rate']= df_base['Delinquency rate'].replace('-', np.nan).replace(' ', np.nan)
df_base['Delinquency rate'].fillna(0 , inplace = True )
df_base['Delinquency rate']= df_base['Delinquency rate'].apply(lambda x : round(x,2)*100).astype(float)
df_base['Delinquency rate'].unique()

array([0.])

In [30]:
df_base["Add'l Non CSC Competitor Info"].fillna("XXX" , inplace = True)
df_base['Other Notes'].fillna("XXX" , inplace = True)

In [31]:
df_base['Sales assignemnt'].fillna('Sales-unknown' , inplace = True)
#df['% business liens have name mismatched'].fillna(0 , inplace = True)
#df['% business debtor entity status showed "red flag" (non compliant, disolved, inactive)'].fillna(0 , inplace = True)
#df.drop(columns= ['Prioritized?','Delivered?','Batch Name'],axis=1 , inplace = True)
df_base.isna().sum()

Index                                                  0
Client (PAG)                                        1547
Parent name (per 2018 PR)                              0
Parent name (per 2019 PR)                              0
Public Record 2019                                     0
Public Record 2018                                     0
% change 2019 vs. 2018                                 0
SF share of public filing                              0
WKLS share of public filing                            0
% of filing using non WKLS vendors                     0
# of filing offices (/ # of filing zips)               0
UCC 3 % over total filing                              0
# of filing jurisdictions                              0
# of jurisdictions w/ filing > 100                     0
Top vol state                                          0
2nd vol state                                          0
3rd vol state                                          0
Top UCC vol State Srch rating  

In [32]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1547 entries, 0 to 1546
Data columns (total 36 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Index                                             1547 non-null   object 
 1   Client (PAG)                                      0 non-null      float64
 2   Parent name (per 2018 PR)                         1547 non-null   object 
 3   Parent name (per 2019 PR)                         1547 non-null   object 
 4   Public Record 2019                                1547 non-null   int32  
 5   Public Record 2018                                1547 non-null   int32  
 6   % change 2019 vs. 2018                            1547 non-null   int32  
 7   SF share of public filing                         1547 non-null   int32  
 8   WKLS share of public filing                       1547 non-null   int32  
 9   % of filing using n

In [33]:
df_base.head()

,Index,Client (PAG),Parent name (per 2018 PR),Parent name (per 2019 PR),Public Record 2019,Public Record 2018,% change 2019 vs. 2018,SF share of public filing,WKLS share of public filing,% of filing using non WKLS vendors,# of filing offices (/ # of filing zips),UCC 3 % over total filing,# of filing jurisdictions,# of jurisdictions w/ filing > 100,Top vol state,2nd vol state,...,% of filing vol in top 6 states (how disbursed?),Self Filer Segments,Updated User type,Asset size (M),Commercial loan value (M),Commercial loans %,Consumer loans %,Real estate loan %,Auto (New & used) loan %,Delinquency rate,Personal Lien %,Business Lien%,% Filing Date Order > 1,Sales assignemnt,Add'l Non CSC Competitor Info,Other Notes
0,CT-tier2-1,NaN,"Accion East, Inc.","Accion East, Inc.",793,634,25,13,87,0,21,6,35,3,NY,FL,...,85,Others,CT-tier2,0.0,0.0,0,0,0,0,0.0,65,35,20,Raquel Torres,XXX,0
1,CT-tier2-2,NaN,"Afnb Holdings, Inc.","Afnb Holdings, Inc.",1184,1133,5,38,61,1,9,56,10,1,TX,NV,...,99,Bank,CT-tier2,0.0,0.0,0,0,0,0,0.0,37,63,47,Monica Butz,XXX,0
2,CT-tier1-3,NaN,AG Georgia Farm Credit,AG Georgia Farm Credit,2233,2341,-5,5,95,0,64,52,7,1,GA,FL,...,100,FCS,CT-tier1,0.0,0.0,0,0,0,0,0.0,94,6,69,Mary Beth Sommer,XXX,0
3,Self-4,NaN,AGCO Corporation,AGCO Corporation,17175,16942,1,99,1,0,143,20,50,39,TX,NC,...,27,Equipment Finance,Self,0.0,0.0,0,0,0,0,0.0,82,18,56,Julee Freeland,XXX,0
4,CT-tier1-5,NaN,Agheritage Farm Credit Services Aca,Agheritage Farm Credit Services Aca,868,796,9,1,99,0,8,50,6,1,AR,DE,...,100,FCS,CT-tier1,0.0,0.0,0,0,0,0,0.0,80,20,63,Mary Beth Sommer,XXX,0


In [34]:
df_base = df_base.rename(columns= {'Public Record 2019 ': 'Public Record 2016', '% change 2019 vs. 2018': '% change 2017 vs. 2016', 
                    'Public Record 2018': 'Public Rdcord 2018', 'Public Record 2019 ':'Public Record 2016'})

In [35]:
df_base.shape

(1547, 36)

In [36]:
%%time
df_master= df_base
print(df_master['Sales assignemnt'].nunique())
print(df_master.shape)

df_master['Access_all_1'] = 'Sheetal Kamani'
df_master['Access_all_2'] = 'Sharyn Shirley'
df_master['Access_all_3'] = 'Shu Morris'
df_master['Access_all_4'] = 'Alexis Jacobson'
df_master['Access_all_5'] = 'Andrew Hoechstetter'
df_master['Access_all_6'] = 'Kochpatchara Pramnomai'

30
(1547, 36)
Wall time: 2.99 ms


In [37]:
print(df_role.shape)
df_role.columns

(27, 4)


Index(['Sale assigned', 'Role1', 'Role2', 'Role3'], dtype='object')

In [38]:
df_merge = df_master[['Parent name (per 2019 PR)','Parent name (per 2018 PR)','Sales assignemnt','Access_all_1','Access_all_2','Access_all_3','Access_all_4','Access_all_5','Access_all_6']]
merged_df= pd.merge(left=df_merge,right=df_role, how='left' , right_on='Sale assigned', left_on='Sales assignemnt')
merged_df

,Parent name (per 2019 PR),Parent name (per 2018 PR),Sales assignemnt,Access_all_1,Access_all_2,Access_all_3,Access_all_4,Access_all_5,Access_all_6,Sale assigned,Role1,Role2,Role3
0,"Accion East, Inc.","Accion East, Inc.",Raquel Torres,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Raquel Torres,Katie Olivier,NaN,NaN
1,"Afnb Holdings, Inc.","Afnb Holdings, Inc.",Monica Butz,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Monica Butz,Joni Baker,NaN,NaN
2,AG Georgia Farm Credit,AG Georgia Farm Credit,Mary Beth Sommer,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Mary Beth Sommer,Donna Kotke,Jennifer Pierroz,Jennifer Watanabe
3,AGCO Corporation,AGCO Corporation,Julee Freeland,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Julee Freeland,Donna Kotke,Jennifer Pierroz,Jennifer Watanabe
4,Agheritage Farm Credit Services Aca,Agheritage Farm Credit Services Aca,Mary Beth Sommer,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Mary Beth Sommer,Donna Kotke,Jennifer Pierroz,Jennifer Watanabe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,"Community Bank Holdings of Texas, Inc.","Community Bank Holdings of Texas, Inc.",Tamela Rice,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Tamela Rice,Katie Olivier,NaN,NaN
1543,Orbian Corporation Ltd.,Orbian Corporation Ltd.,Christina Villa,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Christina Villa,Katie Olivier,NaN,NaN
1544,The First of Long Island Corporation,The First of Long Island Corporation,Leo Ayala,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Leo Ayala,Joni Baker,NaN,NaN
1545,Discover Financial Services,Discover Financial Services,Jerry Kotek,Sheetal Kamani,Sharyn Shirley,Shu Morris,Alexis Jacobson,Andrew Hoechstetter,Kochpatchara Pramnomai,Jerry Kotek,Donna Kotke,Jennifer Pierroz,Jennifer Watanabe


## Code Starts from here

Data_Preparation for the Prospect List and Summary Tab

In [39]:
# df_risk=[]
# with open_xlsb('../input/Risk_Management - reps-v2.xlsb') as wb:
#         with wb.get_sheet('Sum by SF') as sheet:
#             for row in sheet.rows():
#                 df_risk.append([item.v for item in row])
# df_risk = pd.DataFrame(df_risk[1:], columns=df_risk[0])
# df_risk = df_risk.dropna()
# df_risk.to_csv('../input/SumBYSF_base.csv')

# df_risk.shape

In [40]:
# df_examples=[]
# with open_xlsb('../input/Risk_Management - reps-v2.xlsb') as wb:
#         with wb.get_sheet('More Examples') as sheet:
#             for row in sheet.rows():
#                 df_examples.append([item.v for item in row])
# df_examples = pd.DataFrame(df_examples[1:], columns=df_examples[0])
# df_examples= df_examples[cols]
# df_examples.to_csv('../input/Examples_base.csv')

In [41]:
#dictionary_match, dict_rep

In [42]:
dictionary_match= dict(map_match.values.tolist())
dict_rep = dict(zip(rep_assignment['2018 Name'], rep_assignment['Rep (Updated)']))

In [43]:
onlyfiles = [f for f in listdir("../input/Recieved") if isfile(join("../input/Recieved", f))]
sheet_names=[]; file_location=[]; file_names = []
for files in onlyfiles:
    #print(files)
    file_location.append("../input/Recieved/"+ str(files))
    file_names.append(str(files))
    sheet_names.append(files[:4])
file_sheet= dict(zip(file_location, sheet_names))
    
lst_df=[]
for values in file_sheet.keys():
    df=[]
    print(values)
    with open_xlsb(values) as wb:
        with wb.get_sheet(file_sheet[values]) as sheet:
            for row in sheet.rows():
                df.append([item.v for item in row])
    df = pd.DataFrame(df[1:], columns=df[0])
    lst_df.append(df)
    
dict_df= dict(zip(list(file_sheet.keys()), lst_df))
dict_file_names = dict(zip(file_names, lst_df))

../input/Recieved/1037.xlsb
../input/Recieved/1044.xlsb
../input/Recieved/1127.xlsb
../input/Recieved/1136.xlsb
../input/Recieved/1198.xlsb
../input/Recieved/1224.xlsb
../input/Recieved/1299.xlsb
../input/Recieved/1303.xlsb
../input/Recieved/1831.xlsb
../input/Recieved/2290.xlsb
../input/Recieved/2394.xlsb
../input/Recieved/2545.xlsb
../input/Recieved/2554.xlsb
../input/Recieved/2610.xlsb
../input/Recieved/3163.xlsb
../input/Recieved/3164.xlsb
../input/Recieved/3165.xlsb
../input/Recieved/3166.xlsb
../input/Recieved/3167.xlsb
../input/Recieved/3168.xlsb


In [44]:
#dict_file_names

In [45]:
len(dict_file_names)
#len(file_location)

20

In [46]:
count=0
os.chdir(r"C:\Users\Manish.Kumar4\Downloads\WK\Clients\Prospecting Tool\input")
cwd = os.getcwd() 
src = cwd +'\\Recieved\\'
dst = cwd + '\\Processed\\'
for df in dict_file_names.keys():
    count+=1
#    print(count)
    df_temp = dict_file_names[df]
#    print(df_temp)
    df_entity = transform_entity_stat(dict_file_names[df], entity_status )
#    print(df_entity)
    df_= transform_others(dict_file_names[df], dictionary_match, dict_rep)#
#    print(df_)
    sum_by_sf = create_sum_by_sf(df_, df_entity, dict_rep)
#    print(sum_by_sf)
    temp_df, columns = final_calculation(sum_by_sf)
    example_columns, temp_example_df = get_examples(df_entity, dictionary_match, dict_rep)
    if(count==1):
        final_df= pd.DataFrame(columns=list(columns))
        final_example_df= pd.DataFrame(columns=list(example_columns))
    final_df= final_df.append(temp_df)
    final_example_df= final_example_df.append(temp_example_df)[cols]
    shutil.move(os.path.join(src, df), os.path.join(dst, df))

In [47]:
sum_by_sf

,Prospect Name,Filing Sets Count (for Business Liens),JMM total,DNMM total,Entity Status Total,JMM - Need Attention,JMM-OK Filings,JMM-Unresolved Filings,DNMM - Need Attention,DNMM - No Match,DNMM - Unresolved,DNMM - OK Filings,Entity Status - Entities that need attention,Entity Status - Entities dissolved,Entity Status - Unresolved,Entity Status - Active,JMM - Need Attention%,JMM-OK Filings%,JMM-Unresolved Filings%,DNMM - Need Attention%,DNMM - No Match%,DNMM - Unresolved%,DNMM - OK Filings%,Entity Status - Entities that need attention%,Entity Status - Entities dissolved%,Entity Status - Unresolved%,Entity Status - Active%
0,Snap Home Finance Corp,4,4,4,4,0,0,4,0,4,0,0,0,0,4,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [48]:
final_example_df['Date_Processed'] = Date
final_example_df.shape
#final_example_df

(575, 32)

In [49]:
final_df['Date_Processed'] = Date
final_df.shape

(20, 28)

In [50]:
#final_example_df.tail(10) # Check for DNMM Categories codes getting populated

In [51]:
final_df

,Prospect Name,Filing Sets Count (for Business Liens),JMM total,DNMM total,Entity Status Total,JMM - Need Attention,JMM-OK Filings,JMM-Unresolved Filings,DNMM - Need Attention,DNMM - No Match,DNMM - Unresolved,DNMM - OK Filings,Entity Status - Entities that need attention,Entity Status - Entities dissolved,Entity Status - Unresolved,Entity Status - Active,JMM - Need Attention%,JMM-OK Filings%,JMM-Unresolved Filings%,DNMM - Need Attention%,DNMM - No Match%,DNMM - Unresolved%,DNMM - OK Filings%,Entity Status - Entities that need attention%,Entity Status - Entities dissolved%,Entity Status - Unresolved%,Entity Status - Active%,Date_Processed
0,United National Corporation,1159,1195,1203,1200,56,937,202,619,202,0,382,79,73,202,846,0.046862,0.784100,0.169038,0.514547,0.167914,0.0,0.317539,0.065833,0.060833,0.168333,0.705000,Jul-20-2020
0,D. L. Evans Bancorp,5822,5921,6015,5990,82,5148,691,881,691,0,4443,1042,247,691,4010,0.013849,0.869448,0.116703,0.146467,0.114879,0.0,0.738653,0.173957,0.041235,0.115359,0.669449,Jul-20-2020
0,Mercantile Bank Corporation,4526,4578,4622,4632,39,4115,424,3406,424,0,792,571,10,430,3621,0.008519,0.898864,0.092617,0.736910,0.091735,0.0,0.171354,0.123273,0.002159,0.092832,0.781736,Jul-20-2020
0,"New York Community Bancorp, Inc.",35989,37322,38639,37209,21416,11428,4478,7207,4478,0,26954,2929,11,4789,29480,0.573817,0.306200,0.119983,0.186521,0.115893,0.0,0.697585,0.078718,0.000296,0.128705,0.792281,Jul-20-2020
0,AGCO Corporation,34325,34720,34986,34695,2489,24358,7873,6350,7873,0,20763,2814,737,7893,23251,0.071688,0.701555,0.226757,0.181501,0.225033,0.0,0.593466,0.081107,0.021242,0.227497,0.670154,Jul-20-2020
0,Medallion Financial Corp.,14341,14872,15076,14867,8534,3346,2992,3750,2992,0,8334,3083,727,3024,8033,0.573830,0.224987,0.201183,0.248740,0.198461,0.0,0.552799,0.207372,0.048900,0.203404,0.540324,Jul-20-2020
0,Family Security Credit Union Inc,33,33,33,33,0,1,32,1,32,0,0,0,0,32,1,0.000000,0.030303,0.969697,0.030303,0.969697,0.0,0.000000,0.000000,0.000000,0.969697,0.030303,Jul-20-2020
0,United Midwest Savings Bank,2351,2436,2460,2407,246,1923,267,247,267,0,1946,168,32,271,1936,0.100985,0.789409,0.109606,0.100407,0.108537,0.0,0.791057,0.069796,0.013295,0.112588,0.804321,Jul-20-2020
0,Lake Trust Credit Union,228,229,231,231,0,215,14,152,14,0,65,10,1,14,206,0.000000,0.938865,0.061135,0.658009,0.060606,0.0,0.281385,0.043290,0.004329,0.060606,0.891775,Jul-20-2020
0,Technology Credit Union,314,325,330,326,2,114,209,33,209,0,88,12,3,209,102,0.006154,0.350769,0.643077,0.100000,0.633333,0.0,0.266667,0.036810,0.009202,0.641104,0.312883,Jul-20-2020


In [52]:
df_sumbysf= pd.read_csv('../input/SumBYSF_base.csv')
df_examples = pd.read_csv('../input/Examples_base.csv')
#df_sumbysf['Date_Processed'] = datetime.datetime(2020, 5, 25)
df_examples.shape , df_sumbysf.shape

((120386, 32), (715, 30))

In [55]:
df_sumbysf.head()

,Prospect Name,Selected?,Rep Assigned,Filing Sets Count (for Business Liens),JMM total,DNMM total,Entity Status Total,JMM - Need Attention,JMM-OK Filings,JMM-Unresolved Filings,DNMM - Need Attention,DNMM - No Match,DNMM - Unresolved,DNMM - OK Filings,Entity Status - Entities that need attention,Entity Status - Entities dissolved,Entity Status - Unresolved,Entity Status - Active,JMM - Need Attention%,JMM-OK Filings%,JMM-Unresolved Filings%,DNMM - Need Attention%,DNMM - No Match%,DNMM - Unresolved%,DNMM - OK Filings%,Entity Status - Entities that need attention%,Entity Status - Entities dissolved%,Entity Status - Unresolved%,Entity Status - Active%,Date_Processed
0,Banco Santander SA,0.0,Leo Ayala,86649,91869,95065,92103,18826,41372,31671,22901,31671,0,40493,8199,9435,32089,42380,0.204922,0.450337,0.344741,0.240898,0.333151,0.0,0.425951,0.089020,0.102440,0.348403,0.460137,5/25/2020
1,"Security Signal Devices, Inc.",0.0,Christina Villa,3462,3513,3517,3517,116,1165,2232,880,2232,0,405,287,148,2234,848,0.033020,0.331625,0.635354,0.250213,0.634632,0.0,0.115155,0.081604,0.042081,0.635200,0.241115,5/25/2020
2,Union Bank and Trust Company,0.0,Jacob Broussard,6491,6956,7044,6946,1729,3451,1776,1477,1776,0,3791,1124,32,1781,4009,0.248562,0.496118,0.255319,0.209682,0.252129,0.0,0.538189,0.161820,0.004607,0.256407,0.577167,5/25/2020
3,"SEVEN & I HOLDINGS CO., LTD.",0.0,Monica Butz,5885,5932,5937,5934,313,4653,966,1067,966,0,3904,559,306,968,4101,0.052765,0.784390,0.162846,0.179720,0.162708,0.0,0.657571,0.094203,0.051567,0.163128,0.691102,5/25/2020
4,"Limestone Bancorp, Inc.",0.0,Sharon Wilhoite,1187,1207,1234,1218,52,985,170,339,170,0,725,392,6,170,650,0.043082,0.816073,0.140845,0.274716,0.137763,0.0,0.587520,0.321839,0.004926,0.139573,0.533662,5/25/2020


In [56]:
df_examples.head()

,Prospect Name,PROSPECT_ID,FILING_ID,SEC_PARTY_CMN_NM,UCC_FILING_STATE_CD,FILING_NUM,FILING_DT_TM,DEBTOR_CMN_NM,UCC_PERM_ID,MATCH_TYPE,NM_MATCH_FLG,DNMM_CATEGORY,CHT_PERM_ID,BE_ID,CORP_NUM,CHT_FILING_STATE_CD,INCORPORATION_STATE_CD,BE_NM,STD_BUS_NM_TYPE_CD,JMM_CATEGORY,SRC_BE_CUR_STS_CD,ENTITY_STAT,ENTITY_STATUS_CATEGORY,ENTITY_STATUS_CATEGORY_ID,SRC_CORP_TYPE_CD,ENTITY_TYPE,DNMM categories 2,Entity status categories 2,Rep assigned,Filing Year,Filing Month,Date_Processed
0,"Central Bancompany, Inc.",1006.0,212796282.0,CENTRAL BANK OF ST. LOUIS,MO,1801029814542.0,2018010214:37,"KORLISS, LLC",58120280.0,COMP_NM,3.0,MATCH,58120280.0,249513018.0,FL001412038,MO,NV,"KORLISS, LLC",3.0,JURISDICTION MISMATCH,A,ACTIVE,ACTIVE,1.0,8.0,LIMITED LIABILITY COMPANY,MATCH,Active,Jacob Broussard,2018.0,1.0,NaN
1,"Central Bancompany, Inc.",1006.0,213073524.0,CENTRAL BANK OF LAKE OF THE OZARKS,MO,20060004815H,2006011212:12,"FOLSOM RIDGE, LLC",32722350.0,COMP_NM,3.0,MATCH,32722350.0,249120337.0,FL0019414,MO,CO,"FOLSOM RIDGE, LLC",3.0,JURISDICTION MISMATCH,A,ACTIVE,ACTIVE,1.0,8.0,LIMITED LIABILITY COMPANY,MATCH,Active,Jacob Broussard,2006.0,1.0,NaN
2,"Central Bancompany, Inc.",1006.0,212915051.0,CENTRAL BANK OF LAKE OF THE OZARKS,MO,20060004846A,2006011212:25,"FOLSOM RIDGE, LLC",32722350.0,COMP_NM,3.0,MATCH,32722350.0,249120337.0,FL0019414,MO,CO,"FOLSOM RIDGE, LLC",3.0,JURISDICTION MISMATCH,A,ACTIVE,ACTIVE,1.0,8.0,LIMITED LIABILITY COMPANY,MATCH,Active,Jacob Broussard,2006.0,1.0,NaN
3,"Central Bancompany, Inc.",1006.0,238775664.0,BANK OF JACOMO,FL,200406033429.0,20040130.0,"SDI OF LONGWOOD, FLORIDA, LLC",63110416.0,COMP_NM,3.0,MATCH,63110416.0,175141820.0,M07000001143,FL,MO,"SDI OF LONGWOOD, FLORIDA, LLC",3.0,JURISDICTION MISMATCH,A,ACTIVE,ACTIVE,1.0,L2,FOREIGN LIMITED LIABILITY,MATCH,Active,Jacob Broussard,2004.0,1.0,NaN
4,"Central Bancompany, Inc.",1006.0,212982463.0,CENTRAL BANK,MO,1601256555705.0,2016012512:48,COY'S AUTO REPAIR INC,62871482.0,COMP_NM,3.0,MATCH,62871482.0,249123075.0,956741.0,MO,MO,COY'S AUTO REPAIR INC,3.0,MATCH,E,ADMIN DISSOLVED PROFIT,ADMIN DISSOLVED PROFIT,NaN,4.0,GENERAL BUSINESS,MATCH,Need Attention,Jacob Broussard,2016.0,1.0,NaN


In [57]:
print(df_examples.shape, df_sumbysf.shape)
df_examples.drop(df_examples.filter(regex="Unname"),axis=1, inplace=True)
df_sumbysf.drop(df_sumbysf.filter(regex="Unname"),axis=1, inplace=True)
print(df_examples.shape, df_sumbysf.shape)

(120386, 32) (715, 30)
(120386, 32) (715, 30)


In [58]:
df_sumbysf= df_sumbysf.append(final_df, sort= False)  
print(df_sumbysf.shape)
df_examples = df_examples.append(final_example_df, sort= False)

(735, 30)


In [59]:
print(df_examples.shape)

(120961, 32)


In [60]:
print(df_sumbysf.shape)
df_sumbysf = df_sumbysf.drop_duplicates(subset = ['Prospect Name','Date_Processed'], keep='last') 
df_sumbysf = df_sumbysf.sort_values(by=['Date_Processed'])
df_sumbysf = df_sumbysf.drop_duplicates(subset = ['Prospect Name'], keep='last') 
print(df_sumbysf.shape)

(735, 30)
(733, 30)


In [61]:
df_sumbysf.to_csv('../output/SumBYSF_base_{}.csv'.format(Date) , index = False)
df_sumbysf.to_csv('../input/SumBYSF_base.csv', index = False)

df_examples.to_csv('../output/Examples_base_{}.csv'.format(Date), index = False)
df_examples.to_csv('../input/Examples_base.csv', index = False)

In [62]:
print(df_sumbysf.shape)

(733, 30)


#########################################################################3

In [63]:
df_sumbysf= pd.read_csv(r'../input/SumBYSF_base.csv') 
df_sumbysf.drop(df_sumbysf.filter(regex="Unname"),axis=1, inplace=True)
df_sumbysf['Rep Assigned']= ''
df_sumbysf['Selected?']= ''
df_sumbysf.shape

(733, 30)

In [64]:
summary = df_sumbysf.melt(id_vars=['Prospect Name', 'Selected?', 'Date_Processed', 'Rep Assigned'], var_name='KPI', value_name='Value')
summary = summary[['Prospect Name', 'Rep Assigned','KPI','Value']]
summary['Value'] = summary['Value'].apply(lambda x: round(x,2)).astype(float)
summary

,Prospect Name,Rep Assigned,KPI,Value
0,Banco Santander SA,,Filing Sets Count (for Business Liens),86649.00
1,Amboy Bancorporation,,Filing Sets Count (for Business Liens),3036.00
2,Bank First National Corporation,,Filing Sets Count (for Business Liens),5387.00
3,"Commercial Bancgroup, Inc.",,Filing Sets Count (for Business Liens),1393.00
4,1ST GLOBAL CAPITAL LLC,,Filing Sets Count (for Business Liens),737.00
...,...,...,...,...
19053,STATE BANK OF TEXAS,,Entity Status - Active%,0.43
19054,Security Star Bancshares Inc,,Entity Status - Active%,0.56
19055,"Republic Bancorp, Inc.",,Entity Status - Active%,0.47
19056,Koninklijke Philips N.V.,,Entity Status - Active%,0.63


In [65]:
summary['Prospect_Name_lower']= summary['Prospect Name'].str.lower()
merged_df['Parent_Name_2019_lower']= merged_df['Parent name (per 2019 PR)'].str.lower()
merged_df['Parent_Name_2018_lower']= merged_df['Parent name (per 2018 PR)'].str.lower()

In [66]:
summary_2019 = pd.merge(left=merged_df, right= summary , right_on='Prospect_Name_lower', left_on='Parent_Name_2019_lower', how='inner')
li_ = list(set(list(summary['Prospect_Name_lower']))- set(list(summary_2019['Parent_Name_2019_lower'])))

In [67]:
summary_2019['Base_Prospect_Name'] = summary_2019['Parent name (per 2019 PR)']

In [68]:
summary_temp = summary[summary['Prospect_Name_lower'].isin(li_)]
summary_temp.shape

(1690, 5)

In [69]:
summary_2018 = pd.merge(left=merged_df, right= summary_temp , right_on='Prospect_Name_lower', left_on='Parent_Name_2018_lower', how='inner')
summary_2018.shape
summary_2018['Base_Prospect_Name'] = summary_2018['Parent name (per 2018 PR)']

In [70]:
#summary_2018.shape

In [71]:
summary_fin = pd.concat([summary_2019, summary_2018])

In [72]:
summary_fin.shape
#summary_fin.drop_duplicates()
#summary_fin[summary_fin['Parent name (per 2019 PR)'].str.contains('Carbucks of Carolina, Inc.')]

(18876, 21)

In [73]:
summary_fin = summary_fin.drop(columns=['Prospect_Name_lower', 'Parent_Name_2019_lower', 'Parent_Name_2018_lower'])
summary_fin.shape

(18876, 18)

In [74]:
#summary1[summary1['Prospect Name'].str.contains('Old Second Bancorp, Inc.')]
#summary1[summary1['Prospect Name'].str.contains('NEW EQUIPMENT LEASING, INC.')]
summary_fin['Prospect Name'] =  summary_fin['Parent name (per 2018 PR)']
#summary1.columns
#New Equipment Leasing, Inc.

In [75]:
summary_fin.shape

(18876, 18)

In [76]:
summary_fin = summary_fin.rename(columns= {'Public Record 2019 ': 'Public Record 2016', '% change 2019 vs. 2018': '% change 2017 vs. 2016', 
                    'Public Record 2018': 'Public Rdcord 2018', 'Public Record 2019 ':'Public Record 2016'})

In [77]:
#summary_2019
summary_fin.to_csv(r"../output/summary.csv" )

####################################

In [78]:
df_kpi = df_sumbysf[['Prospect Name', 'Rep Assigned',
       'Filing Sets Count (for Business Liens)','JMM - Need Attention%','DNMM - Need Attention%','Entity Status - Entities that need attention%','Entity Status - Entities dissolved%']]

In [79]:
df_kpi['Filing Sets Count (for Business Liens)'] = df_kpi['Filing Sets Count (for Business Liens)'].apply(lambda x: round(x,0)).astype(int)
df_kpi['JMM - Need Attention%'] = df_kpi['JMM - Need Attention%'].apply(lambda x: round(x*100,2)).astype(float)
df_kpi['DNMM - Need Attention%'] = df_kpi['DNMM - Need Attention%'].apply(lambda x: round(x*100,2)).astype(float)
df_kpi['Entity Status - Entities that need attention%'] = df_kpi['Entity Status - Entities that need attention%'].apply(lambda x: round(x*100,2)).astype(float)
df_kpi['Entity Status - Entities dissolved%'] = df_kpi['Entity Status - Entities dissolved%'].apply(lambda x: round(x*100,2)).astype(float)
df_kpi['JMM - Need Attention%'] = df_kpi['JMM - Need Attention%'].apply(lambda x: round(x,2)).astype(float)
#df_kpi['Prospect Name']= df_kpi['Prospect Name'].str.lower()

In [80]:
df_kpi

,Prospect Name,Rep Assigned,Filing Sets Count (for Business Liens),JMM - Need Attention%,DNMM - Need Attention%,Entity Status - Entities that need attention%,Entity Status - Entities dissolved%
0,Banco Santander SA,,86649,20.49,24.09,8.90,10.24
1,Amboy Bancorporation,,3036,1.35,0.99,0.82,0.00
2,Bank First National Corporation,,5387,5.74,25.69,11.45,6.54
3,"Commercial Bancgroup, Inc.",,1393,5.38,28.12,12.60,4.91
4,1ST GLOBAL CAPITAL LLC,,737,6.20,35.97,18.36,7.18
...,...,...,...,...,...,...,...
728,STATE BANK OF TEXAS,,1647,6.55,23.85,21.97,7.51
729,Security Star Bancshares Inc,,3183,1.10,24.95,17.06,5.89
730,"Republic Bancorp, Inc.",,13192,3.14,19.63,30.89,2.37
731,Koninklijke Philips N.V.,,12892,15.17,16.82,11.43,3.67


In [81]:
#df_merge = df_master[['Parent name (per 2018 PR)','Sales assignemnt','Access_all_1','Access_all_2','Access_all_3','Access_all_4','Access_all_5']]
#merged_df= pd.merge(left=df_merge,right=df_role, how='left' , right_on='Sale assigned', left_on='Sales assignemnt')
df_kpi
df_prospects['Prospect_lower']= df_prospects['Prospect'].str.lower()
df_kpi['Prospect_Name_lower']= df_kpi['Prospect Name'].str.lower()
df_kpi = pd.merge(left=df_kpi,right=df_prospects, how='left' , right_on='Prospect_lower', left_on='Prospect_Name_lower')
df_kpi.shape
#df_kpi

(733, 27)

In [82]:
df_master.shape

(1547, 42)

In [83]:
#df_kpi['Prospect_Name_lower']= df_kpi['Prospect Name'].str.lower()
#df_master['Parent_Name_lower']= df_master['Parent name (per 2018 PR)'].str.lower()
df_master['Parent_Name_2018_lower']= df_master['Parent name (per 2018 PR)'].str.lower()
df_master['Parent_Name_2019_lower']= df_master['Parent name (per 2019 PR)'].str.lower()

In [84]:
from datetime import date
df_kpi['Updaed Date'] =  date.today()
final = pd.merge(left=df_master, right= df_kpi , right_on='Prospect_Name_lower', left_on='Parent_Name_2019_lower', how='inner')
final['Base_Prospect_Name'] = final['Parent name (per 2019 PR)']
final.shape

(671, 73)

In [85]:
df_master.columns

Index(['Index', 'Client (PAG)', 'Parent name (per 2018 PR)',
       'Parent name (per 2019 PR)', 'Public Record 2016', 'Public Rdcord 2018',
       '% change 2017 vs. 2016', 'SF share of public filing',
       'WKLS share of public filing', '% of filing using non WKLS vendors',
       '# of filing offices (/ # of filing zips)', 'UCC 3 % over total filing',
       '# of filing jurisdictions', '# of jurisdictions w/ filing > 100',
       'Top vol state', '2nd vol state', '3rd vol state',
       'Top UCC vol State Srch rating', '2nd UCC vol State Srch rating',
       '3rd UCC vol State Srch rating',
       '% of filing vol in top 6 states (how disbursed?)',
       'Self Filer Segments', 'Updated User type', 'Asset size (M)',
       'Commercial loan value (M)', 'Commercial loans %', 'Consumer loans %',
       'Real estate loan %', 'Auto (New & used) loan %', 'Delinquency rate',
       'Personal Lien %', 'Business Lien%', '% Filing Date Order > 1',
       'Sales assignemnt', 'Add'l Non CSC 

In [86]:
#temp

In [87]:
list_= list(final['Prospect_Name_lower'])
df_temp = df_master[~df_master['Parent_Name_2019_lower'].isin(list_)]
final_temp = pd.merge(left=df_temp, right= df_kpi , right_on='Prospect_Name_lower', left_on='Parent_Name_2018_lower', how='inner')
final_temp['Base_Prospect_Name'] = final_temp['Parent name (per 2018 PR)']
final_temp.shape

(53, 73)

In [88]:
list_temp= list(final_temp['Prospect_Name_lower'])
#li_ = list(set(list(df_master['Parent_Name_2019_lower']))- set(list(final['Prospect_Name_lower'])))
df_temp_fin = df_temp[~df_temp['Parent_Name_2018_lower'].isin(list_temp)]
base_prospect_name=[]
for line in df_temp_fin[['Parent name (per 2019 PR)', 'Parent name (per 2018 PR)']].itertuples():
    if line[1] != '':
        base_prospect_name.append(line[1])
    elif line[2] != '':
        base_prospect_name.append(line[2])
    else:
        base_prospect_name.append('Unknown')

df_temp_fin['Base_Prospect_Name'] = np.asarray(base_prospect_name)
#final_fin = pd.merge(left=df_temp_fin, right= df_kpi , right_on='Prospect_Name_lower', left_on='Parent_Name_2018_lower', how='inner')
df_temp_fin.shape

(823, 45)

In [89]:
kpi_fin = pd.concat([final, final_temp, df_temp_fin])

In [90]:
kpi_fin.shape

(1547, 73)

In [91]:
kpi_fin['Prospect Name'] = kpi_fin['Prospect Name'].fillna('')

In [92]:
#final['Sales assignemnt']=final['Sales assignemnt'].str.lower()
#df_role['Sale assigned']=df_role['Sale assigned'].str.lower()
final= pd.merge(left=kpi_fin,right=df_role, how='left' , right_on='Sale assigned', left_on='Sales assignemnt')
final.shape

(1547, 77)

In [93]:
# df_prospects.shape
# #df_prospects['Prospect'] = df_prospects['Prospect'].str.lower()
# final_with_seg = pd.merge(left=final,right=df_prospects, how='left' , right_on='Prospect_Name_lower', left_on='Parent_Name_lower') 
# final_with_seg.shape


In [94]:
final.columns
len(final['Base_Prospect_Name'])- final['Base_Prospect_Name'].nunique()

4

In [95]:
final_with_seg = final.drop(columns=['Prospect_Name_lower', 'Prospect_lower', 'Parent_Name_2018_lower', 'Parent_Name_2019_lower'])

In [96]:
final_with_seg = final_with_seg.rename(columns= {'Public Record 2019 ': 'Public Record 2016', '% change 2019 vs. 2018': '% change 2017 vs. 2016', 
                    'Public Record 2018': 'Public Rdcord 2018', 'Public Record 2019 ':'Public Record 2016'})

In [97]:
final_with_seg_2019 = final_with_seg.rename(columns= {'Public Record 2016':'Public Record 2019 ', '% change 2017 vs. 2016':'% change 2019 vs. 2018' 
                    ,'Public Record 2016':'Public Record 2019 '})

In [98]:
final_with_seg.shape

(1547, 73)

In [99]:
final_with_seg.to_csv(r"../output/KPI.csv" )
final_with_seg_2019.to_csv(r"../output/KPI_2019.csv" )

# More Examples

In [100]:
df_examples = pd.read_csv('../input/Examples_base.csv') 
df_examples.shape

(120961, 32)

In [101]:
df_examples.drop(df_examples.filter(regex="Unname"),axis=1, inplace=True)
df_examples.shape

(120961, 32)

In [102]:
def to_camel(word):
        import re
        return ' '.join(x.capitalize() or '_' for x in word.split(' '))

In [103]:
list_dnmm=[]; list_entity=[]
for line in df_examples[['DNMM categories 2', 'Entity status categories 2']].itertuples():
    list_dnmm.append(to_camel(str(line[1])))
    list_entity.append(to_camel(line[2]))
df_examples['DNMM categories 2'] = np.asarray(list_dnmm)
df_examples['Entity status categories 2'] = np.asarray(list_entity)

In [104]:
df_examples['Prospect_Name_lower']= df_examples['Prospect Name'].str.lower()
#merged_df['Parent_Name_lower']= merged_df['Parent name (per 2018 PR)'].str.lower()

In [105]:
merged_df['Parent_Name_2019_lower']= merged_df['Parent name (per 2019 PR)'].str.lower()
merged_df['Parent_Name_2018_lower']= merged_df['Parent name (per 2018 PR)'].str.lower()

In [106]:
merged_df.shape

(1547, 15)

In [107]:
df_examples_temp = pd.merge(left=merged_df, right= df_examples , right_on='Prospect_Name_lower', left_on='Parent_Name_2019_lower', how='inner')
df_examples_temp['Base_Prospect_Name'] = df_examples_temp['Parent name (per 2019 PR)']
df_examples_temp.shape

(111877, 49)

In [108]:
li_ = list(set(list(df_examples['Prospect_Name_lower'])) - set(list(df_examples_temp['Parent_Name_2019_lower'])))
len(li_)

67

In [109]:
df_temp = df_examples[df_examples['Prospect_Name_lower'].isin(li_)]
df_temp.shape

(9235, 33)

In [110]:
df_examples_temp_2 = pd.merge(left=merged_df, right= df_temp , right_on='Prospect_Name_lower', left_on='Parent_Name_2018_lower', how='inner')
df_examples_temp_2['Base_Prospect_Name'] = df_examples_temp_2['Parent name (per 2018 PR)']
df_examples_temp_2.shape

(3707, 49)

In [111]:
df_examples = pd.concat([df_examples_temp, df_examples_temp_2])
df_examples.shape

(115584, 49)

In [112]:
df_examples['Filing Month'] =df_examples['Filing Month'].fillna('') 

In [113]:
#df_examples['FILING_NUM'] = df_examples['FILING_NUM'].apply(lambda x: int(x))

In [114]:
# df_examples['FILING_NUM'] = df_examples['FILING_NUM'].apply(lambda x: str(x))
# df_examples['Filing Month'] = df_examples['Filing Month'].apply(lambda x: str(x))
# df_examples['NM_MATCH_FLG'] = df_examples['NM_MATCH_FLG'].apply(lambda x: str(x))

df_examples['FILING_NUM'] = df_examples['FILING_NUM'].apply(lambda x: "'" + str(x) + "'")
df_examples['Filing Month'] = df_examples['Filing Month'].apply(lambda x: "'" + str(x) + "'")
df_examples['NM_MATCH_FLG'] = df_examples['NM_MATCH_FLG'].apply(lambda x: "'" + str(x) + "'")

# df_examples['FILING_NUM'] = df_examples['FILING_NUM'].astype(str)
# df_examples['Filing Month'] = df_examples['Filing Month'].astype(str)
# df_examples['NM_MATCH_FLG'] = df_examples['NM_MATCH_FLG'].astype(str)

In [115]:
#df_examples[df_examples['Parent name (per 2019 PR)'].str.contains('Carbucks')]

In [116]:
df_examples = df_examples.drop(columns=['Parent_Name_2018_lower', 'Parent_Name_2019_lower', 'Prospect_Name_lower'])
df_examples['Prospect Name']= df_examples['Parent name (per 2018 PR)']

In [117]:
df_examples.shape

(115584, 46)

In [118]:
# df_examples = df_examples.rename(columns= {'Public Record 2019 ': 'Public Record 2016', '% change 2019 vs. 2018': '% change 2017 vs. 2016', 
#                     'Public Record 2018': 'Public Rdcord 2018', 'Public Record 2019 ':'Public Record 2016'})

In [120]:
df_examples.to_csv(r"../output/Examples.csv" )

In [128]:
df_examples.shape

(115009, 46)